# Train the Neural Mesh Simplification model

## Set up the environment

### [*only required for remote runs*] Remote environment setup

If you are running this notebook remotely (e.g. Google Colab), you'll want to set up the environment by
* Downloading the repository from GitHub
* Setting up the python environment

If are opening this notebook locally, by running `jupyter lab` from the repository root and the right conda environment activated, the above step is not required.

#### Step 1. Check out the repo
That's where the source code for mesh simplification, along with its dependency definitions and other utilities, lives.

In [ ]:
!git clone https://github.com/gennarinoos/neural-mesh-simplification.git neural-mesh-simplification

#### Step 2. Install python version 3.12 using apt-get

Check the current python version by running the following command. This notebook requires Python 3.12 to run. Either install it via your Notebook environment settings and jump to Step 6 or follow all the steps below.

In [ ]:
!python --version

In [ ]:
!sudo apt-get update
!sudo apt-get install python3.12

#### Step 3. Update alternatives to use the new Python version

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 1
!sudo update-alternatives --config python3

#### Step 4. Install pip and the required packages for the new Python version.

In [ ]:
!rm -f get-pip*.py
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
!python -m pip install ipykernel
!python -m ipykernel install --user --name python3.12 --display-name "Python 3.12"

#### Step 5. Restart and verify
At this point you may need to restart the session, after which you want to verify that `python` is at the right version (`3.12`)

In [ ]:
!python --version

#### Step 6. Upgrade pip and setuptools

In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install --upgrade build

### Set repository as the working directory 
CD into the repository downloaded above

In [ ]:
%cd neural-mesh-simplification

### Package requirements

Depending on whether you are using PyTorch on a CPU or a GPU,
you'll have to use the correct binaries for PyTorch and the PyTorch Geometric libraries. You can install them via:

In [ ]:
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
!pip install torch_cluster==1.6.3 torch_geometric==2.5.3 torch_scatter==2.1.2 torch_sparse==0.6.18 -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

Replace “cu121” with the appropriate CUDA version for your system. If you don't know what is your cuda version, run `nvidia-smi`

Only then you can install the requirements via pip:

In [ ]:
!pip install -r requirements.txt
!pip uninstall -y neural-mesh-simplification
!pip install .

---
## Download the training data
We can use the Hugging Face API to download some mesh data to use for training and evaluation.

In [ ]:
import os
import shutil
from huggingface_hub import snapshot_download

target_folder = "data/raw"
wip_folder = os.path.join(target_folder, "wip")
os.makedirs(wip_folder, exist_ok=True)

# abc_train is really large (+5k meshes), so download just a sample
folder_patterns = ["abc_extra_noisy/03_meshes/*.ply", "abc_train/03_meshes/*.ply"]

# Download
snapshot_download(
    repo_id="perler/ppsurf",
    repo_type="dataset",
    cache_dir=wip_folder,
    allow_patterns=folder_patterns[0],
)

# Move files from wip folder to target folder
for root, _, files in os.walk(wip_folder):
    for file in files:
        if file.endswith(".ply"):
            src_file = os.path.join(root, file)
            dest_file = os.path.join(target_folder, file)
            shutil.copy2(src_file, dest_file)
            os.remove(src_file)

# Remove the wip folder
shutil.rmtree(wip_folder)

## Prepare the data
The downloaded data needs to be prepapared for training. We can use a script in the repository we checked out for that.

In [ ]:
!mkdir -p data/processed
!python scripts/preprocess_data.py

---
## Model Training

When using a GPU, ensure the training is happening on the GPU, and the environment is configured properly.

In [ ]:
import torch
print(torch.cuda.is_available())

!nvcc --version

### Start the training

In [ ]:
!python scripts/train.py --data-path data/processed --config configs/default.yaml